# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234371819652                   -0.50    8.0         
  2   -7.250198592591       -1.80       -1.40    1.0   9.58ms
  3   -7.250868089593       -3.17       -1.86    1.0   9.82ms
  4   -7.250931858961       -4.20       -1.85    2.0   11.8ms
  5   -7.251333437467       -3.40       -2.70    1.0   9.77ms
  6   -7.251338124145       -5.33       -3.14    1.0   9.80ms
  7   -7.251338700190       -6.24       -3.56    2.0   11.5ms
  8   -7.251338790848       -7.04       -4.10    2.0   12.2ms
  9   -7.251338798445       -8.12       -4.72    2.0   11.9ms
 10   -7.251338798668       -9.65       -5.24    1.0   10.1ms
 11   -7.251338798699      -10.50       -5.77    2.0   11.6ms
 12   -7.251338798704      -11.37       -5.93    2.0   12.2ms
 13   -7.251338798704      -12.06       -6.44    1.0   10.4ms
 14   -7.251338798705      -13.08       -6.63    2.0   12.2ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.0581417482260896
[ Info: Arnoldi iteration step 2: normres = 0.5951827952560188
[ Info: Arnoldi iteration step 3: normres = 0.8247193521140938
[ Info: Arnoldi iteration step 4: normres = 0.25892772984347256
[ Info: Arnoldi iteration step 5: normres = 0.3136123947173698
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.52e-02, 3.76e-02, 2.44e-01, 1.92e-01, 2.21e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4609781568315167
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (5.44e-03, 4.97e-02, 4.28e-01, 1.20e-01, 9.00e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08867774857031735
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.57e-04, 7.83e-03, 3.48e-02, 3.80e-02, 6.34e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10217093836658096
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.17e-0